In [7]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Output, Input
import dash_table
import plotly.express as px

In [8]:
# —— 1. 辅助函数：Haversine 计算（公里） ——
def haversine(row):
    lat1, lon1, lat2, lon2 = map(np.radians,
        [row.lat_s, row.lng_s, row.lat_d, row.lng_d])
    dlat, dlon = lat2-lat1, lon2-lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 6371 * 2 * np.arcsin(np.sqrt(a))

# —— 2. 读取数据 ——
# 2.1 子仓→门店
ship_ss = pd.read_csv('data/ship_sub2store.csv')

# 2.2 节点经纬度
nodes = pd.read_csv('data/nodes.csv')[['node_id','lat','lng']]

# 2.3 门店区域
stores = pd.read_csv('data/stores.csv')[['store_name','region']]
stores_meta = stores.rename(columns={'store_name':'dest'})

# 2.4 服务要求
svc_req = pd.read_csv('data/service_requirements.csv')[
    ['region','speed_kmph','service_window','fill_rate']
]

# 2.5 费率
rates = pd.read_csv('data/rates.csv')[['mode','rate_per_km']]

# —— 3. 合并 & 计算指标 ——
df = ship_ss.merge(
    nodes.rename(columns={'node_id':'source','lat':'lat_s','lng':'lng_s'}),
    on='source', how='left'
).merge(
    nodes.rename(columns={'node_id':'dest','lat':'lat_d','lng':'lng_d'}),
    on='dest', how='left'
).merge(
    stores_meta, on='dest', how='left'
).merge(
    svc_req, on='region', how='left'
)

# 3.1 里程 & 时长
df['distance_km']     = df.apply(haversine, axis=1)
# 提取 SLA 时长（小时）
df['service_window_hrs'] = df['service_window'].str.extract(r'(\d+)').astype(float)
df['lead_time_hrs']      = df['distance_km'] / df['speed_kmph']
df['delay_flag']         = df['lead_time_hrs'] > df['service_window_hrs']

# 3.2 成本 & 超支
ltl_rate = rates.loc[rates['mode']=='LTL','rate_per_km'].iloc[0]
df['estimated_cost']    = df['qty'] * df['distance_km'] * ltl_rate
# 定义“成本超支”为：高于 全局平均 + 1σ
threshold = df['estimated_cost'].mean() + df['estimated_cost'].std()
df['cost_overrun_flag'] = df['estimated_cost'] > threshold

# 3.3 汇总至“路线”
df['route'] = df['source'] + '→' + df['dest']
agg = df.groupby('route', as_index=False).agg(
    total_qty         = ('qty','sum'),
    delay_cnt         = ('delay_flag','sum'),
    delay_rate        = ('delay_flag','mean'),
    cost_overrun_cnt  = ('cost_overrun_flag','sum'),
    avg_cost          = ('estimated_cost','mean')
)

# —— 4. 构建 Dash App ——
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("🚨 风险预警看板", style={'textAlign':'center'}),

    # 4.1 告警列表
    dash_table.DataTable(
        id='risk-table',
        columns=[
            {'name':'路线','id':'route'},
            {'name':'发运量','id':'total_qty',        'type':'numeric','format':{'specifier':','}},
            {'name':'延误次数','id':'delay_cnt',      'type':'numeric'},
            {'name':'延误率','id':'delay_rate',       'type':'numeric','format':{'specifier':'.1%'}},
            {'name':'超支次数','id':'cost_overrun_cnt','type':'numeric'},
            {'name':'平均成本','id':'avg_cost',        'type':'numeric','format':{'specifier':',.0f'}}
        ],
        data=agg.to_dict('records'),
        style_cell={'padding':'5px','textAlign':'center'},
        style_header={'backgroundColor':'#f1f1f1','fontWeight':'bold'},
        style_data_conditional=[
            {
                'if': {'filter_query':'{delay_rate} > 0.2'},
                'backgroundColor':'#FFE5E5'
            },
            {
                'if': {'filter_query':'{cost_overrun_cnt} > 0'},
                'backgroundColor':'#FFF5E5'
            }
        ],
        page_size=10,
    ),

    html.Br(),
    # 4.2 延误次数热点
    dcc.Graph(id='delay-chart'),
    # 4.3 成本超支热点
    dcc.Graph(id='cost-chart'),
])

# —— 5. 回调：绘制条形图 ——
@app.callback(
    Output('delay-chart','figure'),
    Output('cost-chart','figure'),
    Input('risk-table','data')
)
def update_charts(rows):
    dff = pd.DataFrame(rows)

    fig_delay = px.bar(
        dff.sort_values('delay_cnt',ascending=False).head(10),
        x='route', y='delay_cnt',
        title="🔴 Top10 延误次数路由",
        labels={'delay_cnt':'延误次数','route':'路线'}
    )
    fig_delay.update_layout(margin={'t':40,'b':80,'l':40,'r':20})

    fig_cost = px.bar(
        dff.sort_values('cost_overrun_cnt',ascending=False).head(10),
        x='route', y='cost_overrun_cnt',
        title="🟠 Top10 成本超支路由",
        labels={'cost_overrun_cnt':'超支次数','route':'路线'}
    )
    fig_cost.update_layout(margin={'t':40,'b':80,'l':40,'r':20})

    return fig_delay, fig_cost

if __name__ == '__main__':
    app.run(debug=True)